# 01 – Data Ingestion & Schema Setup

**Goals:**  
- Connect to SQLite database with our `schema.sql`  
- Read price CSV(s) from `data/` and compute daily returns  
- Insert those returns into the `returns` table

In [26]:
import os
from sqlalchemy import create_engine

# Define path for the SQLite database file (in the data/ folder)
db_path = os.path.join("..", "data", "portfolio.db")

# Create a SQLAlchemy engine (creates the file if it doesn't exist)
engine = create_engine(f"sqlite:///{db_path}")

print("Engine ready, database file at:", db_path)

Engine ready, database file at: ..\data\portfolio.db


### 1.2 Execute Schema Statements

Our `sql/schema.sql` contains two statements (`DROP TABLE…;` and `CREATE TABLE…;`).  
SQLite only executes one SQL statement at a time, so we’ll split on `;` and run each.

In [21]:
import os

# 1.2 Load the DDL file
schema_path = os.path.join("..", "sql", "schema.sql")
with open(schema_path, "r") as f:
    ddl = f.read()

# 1.2 Execute each statement separately
with engine.begin() as conn:
    for stmt in ddl.split(";"):
        stmt = stmt.strip()
        if stmt:                      # skip empty lines
            conn.exec_driver_sql(stmt)
print("Schema executed successfully.")

Schema executed successfully.


### 1.3 Verify Table Creation

Check that the `returns` table now exists in our SQLite database.

In [23]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.exec_driver_sql(
        "SELECT name FROM sqlite_master WHERE type='table';"
    )
    tables = [row[0] for row in result]
print("Tables in database:", tables)

Tables in database: ['returns']


# 02 – Read Price CSVs & Compute Daily Returns  

In [28]:
# Install yfinance
%pip install yfinance --quiet

Note: you may need to restart the kernel to use updated packages.


In [29]:
# 2.0 Fetch data via yfinance
import os
import yfinance as yf

tickers = ["^GSPC", "AAPL", "TD"]
out_dir = os.path.join("..", "data")
os.makedirs(out_dir, exist_ok=True)

for ticker in tickers:
    # Download daily data
    df = yf.download(ticker, start="2018-01-01", end="2023-12-31", progress=False)
    # Save to CSV
    filename = f"{ticker.replace('^','')}.csv"  # removes ^ from filename
    csv_path = os.path.join(out_dir, filename)
    df.to_csv(csv_path)
    print(f"Saved {csv_path}")

C:\Users\User\AppData\Local\Temp\ipykernel_14860\3132212848.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2018-01-01", end="2023-12-31", progress=False)


Saved ..\data\GSPC.csv


C:\Users\User\AppData\Local\Temp\ipykernel_14860\3132212848.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2018-01-01", end="2023-12-31", progress=False)


Saved ..\data\AAPL.csv


C:\Users\User\AppData\Local\Temp\ipykernel_14860\3132212848.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2018-01-01", end="2023-12-31", progress=False)


Saved ..\data\TD.csv


In [30]:
import glob
print(glob.glob(os.path.join("..","data","*.csv")))

['..\\data\\AAPL.csv', '..\\data\\GSPC.csv', '..\\data\\TD.csv']


### 2.1 Load CSVs and Compute Daily Returns

In [31]:
import pandas as pd

# Define your data path
data_path = os.path.join("..", "data")
tickers = ["GSPC", "AAPL", "TD"]

# Empty DataFrame to collect all returns
all_returns = []

for ticker in tickers:
    file_path = os.path.join(data_path, f"{ticker}.csv")
    df = pd.read_csv(file_path, parse_dates=["Date"])
    df = df[["Date", "Adj Close"]].copy()
    df["Ticker"] = ticker
    df["Daily_Return"] = df["Adj Close"].pct_change()
    df.dropna(inplace=True)
    all_returns.append(df[["Date", "Ticker", "Daily_Return"]])

# Combine into one DataFrame
returns_df = pd.concat(all_returns)
returns_df.head()

ValueError: Missing column provided to 'parse_dates': 'Date'

In [32]:
for ticker in tickers:
    file_path = os.path.join(data_path, f"{ticker}.csv")
    df = pd.read_csv(file_path)
    print(f"\n{ticker} columns: {df.columns.tolist()}")



GSPC columns: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']

AAPL columns: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']

TD columns: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']
